<a href="https://colab.research.google.com/github/MustafaGhaziTalab/MustafaTalab_/blob/main/Processing_undergraduate_student_catalogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [23]:
!ls "/content/drive/My Drive/MMM"


student_records.csv


In [24]:
import pandas as pd
from sortedcontainers import SortedDict
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define the path to the CSV file
data_path = '/content/drive/MyDrive/MMM/student_records.csv'

# Load data from CSV into a DataFrame
def load_data(csv_path):
    data = pd.read_csv(csv_path)
    return data

# Function to display the structure of the data
def analyze_data_structure(data):
    print("Column Names:", data.columns.tolist())
    print("First Few Rows of Data:")
    print(data.head())

# Create a B-tree from the data
def create_btree_from_data(data):
    b_tree = SortedDict()
    for index, row in data.iterrows():
        # تحويل 'MATRIC NO' إلى نص قبل استخدامه كمفتاح
        matric_no = str(row['MATRIC NO'])
        b_tree[matric_no] = row.to_dict()
    return b_tree


# Load the data
data = load_data(data_path)

# Analyze the data structure
analyze_data_structure(data)

# Create the B-tree
student_btree = create_btree_from_data(data)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Column Names: ['MATRIC NO', 'NAME STYLE', 'CODE', 'COURSE TITLE', 'LEARNING METHOD', 'SECTION', 'CREDIT STATUS']
First Few Rows of Data:
   MATRIC NO    NAME STYLE      CODE  \
0  MCS000000    JWJMH RPUN  MCSD1013   
1  MCS000001  DEFJAOV FPRM  MCSD1013   
2  MCS000002   KMFAX HMWWG  UECS6023   
3  MCS000003  NOYURXA WDML  MCSD2123   
4  MCS000004    MCIJL LPRZ  MECS1043   

                                 COURSE TITLE LEARNING METHOD  SECTION  \
0         Business Intelligence and Analytics              FC      1.0   
1         Business Intelligence and Analytics              FC      1.0   
2  Introduction to Technical Entrepreneurship              FC      1.0   
3               Big Data Mining and Streaming              FC      1.0   
4    Research Methodology in Computer Science              FC      1.0   

   CREDIT STATUS  
0            3.0  
1         

In [12]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define the path to the CSV file
data_path = '/content/drive/MyDrive/MMM/student_records.csv'

# Load data from CSV into a DataFrame
def load_data(csv_path):
    return pd.read_csv(csv_path)

# Load the data
data = load_data(data_path)

# Create the widgets for the interface
search_input = widgets.Text(placeholder='Enter MATRIC NO or NAME', description='Search:')
add_inputs = {column: widgets.Text() for column in data.columns}
edit_inputs = {column: widgets.Text() for column in data.columns}
delete_input = widgets.Text(description='MATRIC NO:')

search_button = widgets.Button(description='Search', button_style='success')
add_button = widgets.Button(description='Add', button_style='info')
edit_button = widgets.Button(description='Update', button_style='warning')
delete_button = widgets.Button(description='Delete', button_style='danger')

output_area = widgets.Output()

# Function to refresh the output area with new data
def refresh_output(new_data):
    with output_area:
        clear_output()
        display(new_data)

# Define the button click actions
def on_search_clicked(b):
    with output_area:
        clear_output()
        search_query = search_input.value.lower()
        if search_query:
          # You can modify this code to specify the specific columns you want to search
             # Suppose we want to search in both "MATRIC NO" and "NAME" columns
            results = data[
                data['MATRIC NO'].str.contains(search_query, case=False, na=False) |
                data['NAME STYLE'].str.contains(search_query, case=False, na=False)
            ]
            if not results.empty:
                display(results)
            else:
                print("No records found.")
        else:
            print("Please enter a search query.")

def on_add_clicked(b):
    new_data = {key: val.value for key, val in add_inputs.items()}
    if new_data['MATRIC NO'] not in data['MATRIC NO'].values:
        data.loc[len(data)] = new_data
        refresh_output(data)
        for widget in add_inputs.values():
            widget.value = ''  # Clear the input after adding
    else:
        print(f"Matric No {new_data['MATRIC NO']} already exists.")




def on_edit_search_clicked(b):
    with output_area:
        clear_output()
        # Clear previous edit inputs
        for widget in edit_inputs.values():
            widget.value = ''

        search_query = edit_search_input.value
        if search_query:
            # Search by MATRIC NO or NAME
            student_record = data[
                (data['MATRIC NO'].astype(str).str.lower() == search_query.lower()) |
                (data['NAME STYLE'].str.lower().str.contains(search_query.lower()))
            ]

            if not student_record.empty:
                # Assume the first match is the record to edit
                student_info = student_record.iloc[0].to_dict()
                for key, widget in edit_inputs.items():
                    widget.value = str(student_info[key])  # Pre-fill the edit inputs with the student's current information
                display(widgets.VBox([widgets.Label(f"Editing {search_query}"), *edit_inputs.values(), edit_button]))
            else:
                print(f"No student found with search query '{search_query}'.")

edit_search_input = widgets.Text(placeholder='Enter MATRIC NO or NAME', description='Edit Search:')
edit_search_button = widgets.Button(description='Find Student', button_style='success')
edit_search_button.on_click(on_edit_search_clicked)

# Update the existing on_edit_clicked function
def on_edit_clicked(b):
    with output_area:
        clear_output()
        matric_no = edit_inputs['MATRIC NO'].value  # Assuming the matric no is not changed

        # Ensure that the MATRIC NO exists in the DataFrame
        if matric_no in data['MATRIC NO'].values:
            # Get the index of the student in the DataFrame
            student_index = data.index[data['MATRIC NO'] == matric_no].tolist()[0]

            # Update the student's information
            for key, widget in edit_inputs.items():
                if key != 'MATRIC NO':  # Avoid changing the MATRIC NO
                    data.at[student_index, key] = widget.value if widget.value else data.at[student_index, key]

            # Save the updated DataFrame back to CSV
            data.to_csv(data_path, index=False)

            print(f"Student record for MATRIC NO {matric_no} has been updated.")
        else:
            print(f"No student found with MATRIC NO {matric_no}.")

# Create a new section for editing
edit_search_section = widgets.VBox([edit_search_input, edit_search_button])
edit_section = widgets.VBox([edit_search_section, output_area])



# Define the interface before using it to update children
interface = widgets.Tab()

# Define other sections of the interface
search_section = widgets.VBox([search_input, search_button])
add_section = widgets.VBox([widgets.VBox([widgets.Label(key), add_inputs[key]]) for key in data.columns] + [add_button])
edit_section = widgets.VBox([widgets.VBox([widgets.Label(key), edit_inputs[key]]) for key in data.columns] + [edit_button])
delete_section = widgets.VBox([delete_input, delete_button])

# Group sections together in interface
interface.children = [search_section, add_section, edit_section, delete_section]

# Now you can update children safely
children_list = list(interface.children)
children_list[2] = edit_section  # Update the edit section
interface.children = tuple(children_list)  # Reconvert the list to a tuple after modification





# interface.children is a tuple, so we turn it into a list to modify it
children_list = list(interface.children)
children_list[2] = edit_section            # Update the third item in the list, which represents the edit section
interface.children = tuple(children_list)  # Reconvert the list to a tuple after modification



# Now, update the interface to include this new section
#interface.set_title(2, 'Edit')
#interface.children[2] = edit_section



def on_delete_clicked(b):
    matric_no = delete_input.value
    if matric_no in data['MATRIC NO'].values:
        data.drop(data[data['MATRIC NO'] == matric_no].index, inplace=True)
        refresh_output(data)
    else:
        print(f"No student found with MATRIC NO {matric_no}.")



# New functionality to search for a student's courses and delete a specific course
def find_student_courses(matric_no):
    return data[data['MATRIC NO'] == matric_no]

def delete_course(student_courses, course_title):
    return student_courses[student_courses['COURSE TITLE'] != course_title]

# Added new interfaces for deleting courses
course_delete_input = widgets.Text(description='Course Title:')
course_delete_button = widgets.Button(description='Delete Course', button_style='danger')
course_output_area = widgets.Output()

# Function of clicking on the delete session button
def on_course_delete_clicked(b):
    with course_output_area:
        clear_output()
        # Search for a student using student number or name
        search_query = delete_course_input.value
        student_courses = data[
            (data['MATRIC NO'].astype(str).str.lower() == search_query.lower()) |
            (data['NAME STYLE'].str.lower().str.contains(search_query.lower()))
        ]

        # If the student is found, delete the selected course
        if not student_courses.empty and course_delete_input.value:
            updated_courses = delete_course(student_courses, course_delete_input.value)
            print(f"Course '{course_delete_input.value}' has been deleted for student '{search_query}'.")
            # Update key data based on updated cycles
            data.update(updated_courses)
            # Save Changes
            data.to_csv(data_path, index=False)
        else:
            print(f"No student found with search query '{search_query}' or no course title entered.")

# Associate the delete cycle button with the function
course_delete_button.on_click(on_course_delete_clicked)

# Updated the user interface to include a section for deleting courses
course_delete_section = widgets.VBox([delete_course_input, course_delete_input, course_delete_button, course_output_area])

# Bind the buttons to their respective functions
search_button.on_click(on_search_clicked)
add_button.on_click(on_add_clicked)
edit_button.on_click(on_edit_clicked)
delete_button.on_click(on_delete_clicked)

# Arrange the interface
search_section = widgets.VBox([search_input, search_button])
add_section = widgets.VBox([widgets.VBox([widgets.Label(key), add_inputs[key]]) for key in data.columns] + [add_button])
edit_section = widgets.VBox([widgets.VBox([widgets.Label(key), edit_inputs[key]]) for key in data.columns] + [edit_button])
delete_section = widgets.VBox([delete_input, delete_button])

interface = widgets.Tab([search_section, add_section, edit_section, delete_section])
interface.set_title(0, 'Search')
interface.set_title(1, 'Add')
interface.set_title(2, 'Edit')
interface.set_title(3, 'Delete')

# Display the interface
display(interface, output_area)

# Function to delete a specific course for a specific student
def delete_student_course(matric_no, course_title):
    global data  # Use global data
    # Verify the presence of the student and the course
    if matric_no in data['MATRIC NO'].values:
        student_courses = data[data['MATRIC NO'] == matric_no]
        if course_title in student_courses['COURSE TITLE'].values:
            # Delete the course
            data = data[~((data['MATRIC NO'] == matric_no) & (data['COURSE TITLE'] == course_title))]
            #Save the changes to the file
            data.to_csv(data_path, index=False)
            print(f"Course '{course_title}' has been deleted for student with MATRIC NO '{matric_no}'.")
        else:
            print(f"Course title '{course_title}' not found for student with MATRIC NO '{matric_no}'.")
    else:
        print(f"No student found with MATRIC NO '{matric_no}'.")

# Add a user interface to delete a course
delete_course_matric_input = widgets.Text(description='MATRIC NO:')
delete_course_title_input = widgets.Text(description='Course Title:')
delete_course_button = widgets.Button(description='Delete Course', button_style='danger')
delete_course_output = widgets.Output()

# Link the delete course button to the delete function
def on_delete_course_button_clicked(b):
    matric_no = delete_course_matric_input.value.strip()
    course_title = delete_course_title_input.value.strip()
    with delete_course_output:
        clear_output()
        if matric_no and course_title:
            delete_student_course(matric_no, course_title)
        else:
            print("Please enter both MATRIC NO and Course Title.")

delete_course_button.on_click(on_delete_course_button_clicked)

# Add the new interface to the main section
course_delete_section = widgets.VBox([
    delete_course_matric_input,
    delete_course_title_input,
    delete_course_button,
    delete_course_output
])

# Now you can add the Delete Course section to the Tab widget in the main interface
# For example, adding a new book:
tabs_titles = ['Search', 'Add', 'Edit', 'Delete', 'Delete Course']
interface.children = list(interface.children) + [course_delete_section]  # Add the new section
for i, title in enumerate(tabs_titles):
    interface.set_title(i, title)





Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Output()